In [1]:
import numpy as np
import pandas as pd
import pymongo
import json
import datetime as dt
import pickle
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
from sklearn.externals import joblib
import dateutil.parser

# Scrape Data and Organize

In [2]:
# create mongo db connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# connect to my nba data db
db = client.nba_data_db

In [3]:
# drop any items in training data collection
db.training_data.drop()

In [4]:
# Url to get all boxscores for 2017-2018 regular season games
nba_17_18_boxscores_url = "https://stats.nba.com/teams/boxscores-advanced/?Season=2017-18&SeasonType=Regular%20Season"
nba_17_18_fourfactor_url = "https://stats.nba.com/teams/boxscores-four-factors/?Season=2017-18&SeasonType=Regular%20Season"

In [7]:
# create splinter browser
def init_browser(head_bool):
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=head_bool)
    return browser

In [100]:
browser = init_browser(False)

In [8]:
# visit website
browser.visit(nba_17_18_boxscores_url)
time.sleep(3)

In [9]:
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
new_df = df[0].drop(columns=['Season'])
# Rename column names
new_df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp"}, inplace=True)
# change gamedate to datetime
new_df['GameDate'] = new_df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))

In [10]:
# visit website
browser.visit(nba_17_18_fourfactor_url)
time.sleep(3)

In [11]:
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp", "OppFTA\xa0Rate": "OppFTARate"}, inplace=True)
# Convert to float percent values
df['OREB%'] = df['OREB%'].str.rstrip('%').astype('float')
df['OppOREB%'] = df['OppOREB%'].str.rstrip('%').astype('float')
df['OppeFG%'] = df['OppeFG%'].str.rstrip('%').astype('float')
df['eFG%'] = df['eFG%'].str.rstrip('%').astype('float')
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
df = df.drop(['MIN', 'MatchUp', 'W/L', 'eFG%', 'OREB%', 'TOV%'], axis=1)

In [12]:
result = pd.merge(new_df, df, on=['Team', 'GameDate'])
result.columns

Index(['Team', 'MatchUp', 'GameDate', 'W/L', 'MIN', 'OffRtg', 'DefRtg',
       'NetRtg', 'AST%', 'AST/TO', 'ASTRatio', 'OREB%', 'DREB%', 'REB%',
       'TOV%', 'eFG%', 'TS%', 'PACE', 'PIE', 'FTARate', 'OppeFG%',
       'OppFTARate', 'OppTOV%', 'OppOREB%'],
      dtype='object')

In [96]:
nba_18_19_boxscores_url = "https://stats.nba.com/teams/boxscores-advanced/"
nba_18_19_fourfactor_url = "https://stats.nba.com/teams/boxscores-four-factors/"

In [97]:
# drop any items in training data collection
db.testing_data.drop()

In [101]:
# visit website
browser.visit(nba_18_19_boxscores_url)
time.sleep(3)

In [102]:
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
new_df = df[0].drop(columns=['Season'])
# Rename column names
new_df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp"}, inplace=True)
# change gamedate to datetime
new_df['GameDate'] = new_df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))

In [103]:
# visit website
browser.visit(nba_18_19_fourfactor_url)
time.sleep(3)

In [104]:
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp", "OppFTA\xa0Rate": "OppFTARate"}, inplace=True)
# Convert to float percent values
df['OREB%'] = df['OREB%'].str.rstrip('%').astype('float')
df['OppOREB%'] = df['OppOREB%'].str.rstrip('%').astype('float')
df['OppeFG%'] = df['OppeFG%'].str.rstrip('%').astype('float')
df['eFG%'] = df['eFG%'].str.rstrip('%').astype('float')
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
df = df.drop(['MIN', 'MatchUp', 'W/L', 'eFG%', 'OREB%', 'TOV%'], axis=1)

In [105]:
result_new = pd.merge(new_df, df, on=['Team', 'GameDate'])
result_new.columns

Index(['Team', 'MatchUp', 'GameDate', 'W/L', 'MIN', 'OffRtg', 'DefRtg',
       'NetRtg', 'AST%', 'AST/TO', 'ASTRatio', 'OREB%', 'DREB%', 'REB%',
       'TOV%', 'eFG%', 'TS%', 'PACE', 'PIE', 'FTARate', 'OppeFG%',
       'OppFTARate', 'OppTOV%', 'OppOREB%'],
      dtype='object')

# Calculate and Store Data for Web App Use

In [20]:
df_winloss = pd.concat([result, result_new])

In [21]:
df_winloss = df_winloss.groupby(['W/L'])['AST%', 'AST/TO', 'ASTRatio', 'DREB%', 
                                     'DefRtg','OREB%', 'OffRtg', 'PACE', 
                                     'PIE', 'REB%', 'TOV%','TS%',
                                     'eFG%', 'FTARate', 'OppFTARate', 'OppOREB%',
                                     'OppTOV%', 'OppeFG%'].mean()

In [22]:
df_winloss = df_winloss.reset_index()

In [23]:
db.nba_stats_data.drop()

In [24]:
# export to json
items = df_winloss.to_json(orient='records')
# load json string to json
items_db = json.loads(items)

In [25]:
# insert data to collection
db.nba_stats_data.insert_many(items_db)

In [26]:
# export to json
items = result.to_json(orient='records', date_format='iso')
# load json string to json
items_db = json.loads(items)

In [27]:
# insert data to collection
db.training_data.insert_many(items_db)

In [106]:
# export to json
items = result_new.to_json(orient='records', date_format='iso')
# load json string to json
items_db = json.loads(items)

In [107]:
# insert data to collection
db.testing_data.insert_many(items_db)

In [30]:
teams = result_new['Team'].unique()

In [31]:
nbateam = [team for team in teams]

In [32]:
nbateam.sort()

In [34]:
item = {'Teams': nbateam}

In [35]:
db.nba_teams_data.drop()

In [36]:
db.nba_teams_data.insert_one(item)

## Storing Model Params

In [53]:
ffscaler_filename = "fourfactor_scaler.save"
myscaler_filename = "my_scaler.save"
ffmodel_filename = "fourfactor.pkl"
mymodel_filename = "mymodel.pkl"
ff_scaler = joblib.load(ffscaler_filename)
my_scaler = joblib.load(myscaler_filename)
ff_model = pickle.load(open(ffmodel_filename, 'rb'))
my_model = pickle.load(open(mymodel_filename, 'rb'))

In [54]:
ff_params = ['eFG%', 'FTARate', 'TOV%', 'OREB%', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%']

In [55]:
ff_coeff = ff_model.coef_

In [56]:
ff_param_dict = {"Model": 'FourFactor'}

In [57]:
params = {}
for x in range(len(ff_params)):
    params[ff_params[x]] = ff_coeff[0][x]

In [58]:
ff_param_dict['Params'] = params

In [59]:
ff_param_dict

{'Model': 'FourFactor',
 'Params': {'eFG%': 4.9132934560494235,
  'FTARate': 0.9987336863865881,
  'TOV%': -2.227021194165143,
  'OREB%': 2.0183340464234645,
  'OppFTARate': -0.99873368638659,
  'OppOREB%': -2.0183340464234663,
  'OppTOV%': 2.2270211941651405,
  'OppeFG%': -4.9132934560494235}}

In [60]:
db.model_data.drop()

In [61]:
# insert data to collection
db.model_data.insert_one(ff_param_dict)

In [62]:
my_params = ['TS%', 'TOV%', 'OREB%', 'FTARate', 'DefRtg', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%']
my_coeff = my_model.coef_
my_param_dict = {"Model": 'MyModel'}

In [63]:
params = {}
for x in range(len(my_params)):
    params[my_params[x]] = my_coeff[0][x]

In [64]:
my_param_dict['Params'] = params

In [65]:
my_param_dict

{'Model': 'MyModel',
 'Params': {'TS%': 57.69942116623005,
  'TOV%': -26.08780404995324,
  'OREB%': 24.248607829273393,
  'FTARate': -1.1509283544325075,
  'DefRtg': -55.61039438484274,
  'OppFTARate': -1.4081111981218344,
  'OppOREB%': -2.2944071960776307,
  'OppTOV%': 0.5013923099327715,
  'OppeFG%': -3.9724455982781612}}

In [66]:
db.model_data.insert_one(my_param_dict)

# Updating Testing DB

In [3]:
def update_testing_db(df_new):
    # create mongo db connection
    conn = 'mongodb://localhost:27017'
    client = pymongo.MongoClient(conn)
    # connect to my nba data db
    db = client.nba_data_db
    # Find latest boxscore date
    count = db.testing_data.count_documents({})
    if count != 0:
        # Find latest boxscore date
        temp = db.testing_data.find().sort('GameDate',pymongo.DESCENDING).limit(1)
        date_index = temp[0]['GameDate']
        df_update = df_new.loc[df_new['GameDate'] > date_index]
    else:
        date_index = None
        df_update = df_new
    # Get update length
    update_length = len(df_update)
    if update_length > 0:
        print(f'Updating testing_db with {update_length}')
        #for index, row in df_update.iterrows():
            #print(row['MatchUp'])
        items = df_update.to_json(orient='records', date_format='iso')
        # Load to json
        items_db = json.loads(items)
        #print(items)
        # insert data to collection
        db.testing_data.insert_many(items_db)
    else:
        print('No records to update!')
    return update_length

In [4]:
def update_nba_stats():
    # create mongo db connection
    conn = 'mongodb://localhost:27017'
    client = pymongo.MongoClient(conn)
    # connect to my nba data db
    db = client.nba_data_db
    # Get training data
    temp = db.training_data.find()
    temp = list(temp)
    for i in temp:
        i.pop('_id', None)
    df_train = pd.DataFrame(temp)
    # Get training data
    temp = db.testing_data.find()
    temp = list(temp)
    for i in temp:
        i.pop('_id', None)
    df_test = pd.DataFrame(temp)
    df_winloss = pd.concat([df_train, df_test])
    df_winloss = df_winloss.groupby(['W/L'])['AST%', 'AST/TO', 'ASTRatio', 'DREB%', 
                                     'DefRtg','OREB%', 'OffRtg', 'PACE', 
                                     'PIE', 'REB%', 'TOV%','TS%',
                                     'eFG%', 'FTARate', 'OppFTARate', 'OppOREB%',
                                     'OppTOV%', 'OppeFG%'].mean()
    df_winloss = df_winloss.reset_index()
    db.nba_stats_data.drop()
    # export to json
    items = df_winloss.to_json(orient='records')
    # load json string to json
    items_db = json.loads(items)
    # insert data to collection
    db.nba_stats_data.insert_many(items_db)
    return(df_winloss)

In [5]:
nba_18_19_boxscores_url = "https://stats.nba.com/teams/boxscores-advanced/"
nba_18_19_fourfactor_url = "https://stats.nba.com/teams/boxscores-four-factors/"

In [8]:
browser = init_browser(False)

In [9]:
# visit website
browser.visit(nba_18_19_boxscores_url)
time.sleep(3)

In [10]:
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
new_df = df[0].drop(columns=['Season'])
# Rename column names
new_df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp"}, inplace=True)
# change gamedate to datetime
new_df['GameDate'] = new_df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))

In [11]:
# visit website
browser.visit(nba_18_19_fourfactor_url)
time.sleep(3)

In [12]:
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp", "OppFTA\xa0Rate": "OppFTARate"}, inplace=True)
# Convert to float percent values
df['OREB%'] = df['OREB%'].str.rstrip('%').astype('float')
df['OppOREB%'] = df['OppOREB%'].str.rstrip('%').astype('float')
df['OppeFG%'] = df['OppeFG%'].str.rstrip('%').astype('float')
df['eFG%'] = df['eFG%'].str.rstrip('%').astype('float')
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
df = df.drop(['MIN', 'MatchUp', 'W/L', 'eFG%', 'OREB%', 'TOV%'], axis=1)

In [13]:
result_new = pd.merge(new_df, df, on=['Team', 'GameDate'])
result_new.columns

Index(['Team', 'MatchUp', 'GameDate', 'W/L', 'MIN', 'OffRtg', 'DefRtg',
       'NetRtg', 'AST%', 'AST/TO', 'ASTRatio', 'OREB%', 'DREB%', 'REB%',
       'TOV%', 'eFG%', 'TS%', 'PACE', 'PIE', 'FTARate', 'OppeFG%',
       'OppFTARate', 'OppTOV%', 'OppOREB%'],
      dtype='object')

In [14]:
update_testing_db(result_new)
update_nba_stats()

Updating testing_db with 6


C:\Users\charl\Anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



,W/L,AST%,AST/TO,ASTRatio,DREB%,DefRtg,OREB%,OffRtg,PACE,PIE,REB%,TOV%,TS%,eFG%,FTARate,OppFTARate,OppOREB%,OppTOV%,OppeFG%
0,L,57.933540,1.633426,16.318702,72.439408,114.104628,25.952767,102.714933,99.160377,43.083063,48.123664,14.716555,52.972662,49.412786,0.247946,0.269527,27.561641,13.933445,55.241985
1,W,60.085162,1.979690,18.282872,74.048712,102.714933,27.561641,114.104628,99.160377,56.917366,51.877195,13.933445,58.727863,55.241985,0.269527,0.247946,25.952767,14.716555,49.412786


In [115]:
db.testing_data.drop()